# Setup
Import libraries for use with this notebook environment, you do not need these libraries when you write your application code.

In [5]:
import sys
import os
import boto3


# Input parameters
Global constants that help the notebook create Panorama resources on your behalf.

In [6]:
# Device ID, should look like: device-oc66nax4cgzwhyuaeyifrqowue
DEVICE_ID = input( 'DEVICE_ID (format: device-*)' ).strip()

# Enter your S3 bucket info here
S3_BUCKET = input( 'S3_BUCKET' ).strip()

# Enter your desired AWS Panorama region
AWS_REGION = input( 'AWS_REGION (e.g. us-east-1)' ).strip()

# Enter application role to be deployed in panorama device (S3 and kinesis firehose required, secretsmanager and kinesis video are optional if want remote view)
APPLICATION_ROLE = input( 'Application ROLE ARN' ).strip()

# Model name
ML_MODEL_FNAME = 'yolov5m'

DEVICE_ID (format: device-*) device-1
S3_BUCKET eric-panorama-807496703385
AWS_REGION (e.g. us-east-1) us-east-1
Application ROLE ARN arn:aws:iam::807496703385:role/test-multitracker


In [19]:
model_package_name = 'YOLOV5M'
model_data_shape = '{"input0":[1,3,640,640]}'

# application name
app_name = 'multitracker'

## package names and node names
code_package_name = 'multitracker_app'
camera_node_name = 'abstract_rtsp_media_source'

container_asset_name = 'code_asset'

# model node name, raw model path (without platform dependent suffics), and input data shape
model_node_name = "multitracker_model"
model_file_basename = "./models/" + ML_MODEL_FNAME

# video file path to simulate camera stream
# videoname = '../common/test_utility/videos/TownCentreXVID.avi'

sys.path.insert( 0, os.path.abspath( "./test_utility" ) )
import panorama_test_utility

# instantiate boto3 clients
s3_client = boto3.client('s3')
panorama_client = boto3.client('panorama', region_name=AWS_REGION)

# AWS account ID
account_id = boto3.client("sts").get_caller_identity()["Account"]

# Set up application
Every application uses the creator's AWS Account ID as the prefix to uniquely identifies the application resources. Running panorama-cli import-application replaces the generic account Id with your account Id.

In [20]:
!cd ./{app_name} && panorama-cli import-application

Sucessfully imported application


# Import model

Compress .pt model file to .tar.gz

In [21]:
!tar -zcvf {model_file_basename}.tar.gz {model_file_basename}.pt

./models/yolov5m.pt


In [22]:
!aws s3 cp {model_file_basename}.tar.gz s3://{S3_BUCKET}/{app_name}/

upload: models/yolov5m.tar.gz to s3://eric-panorama-807496703385/multitracker/yolov5m.tar.gz


## Prepare model for testing with notebook server

In [25]:
# Compile the model to run with test-utility.
# This step takes 7 mins ~ 10 mins.
%run ./test_utility/panorama_test_utility_compile.py \
\
--s3-model-location s3://{S3_BUCKET}/{app_name}/ \
\
--model-node-name {model_node_name} \
--model-file-basename ./models/{ML_MODEL_FNAME} \
--model-data-shape '{model_data_shape}' \
--model-framework pytorch

Checking [multitracker_model]
Compiled model file [./models/multitracker_model/yolov5m-LINUX_ARM64.tar.gz] doesn't exist. Need compilation.
Uploading [yolov5m.tar.gz] to [s3://eric-panorama-807496703385/multitracker/multitracker_model] ...
Compiling [yolov5m.tar.gz] ...
Created compilation job [Comp-Job20230524125222PM]
Compilation job status : STARTING ...
Compilation job status : INPROGRESS ..
Compilation job status : FAILED 

RuntimeError: Model compilation failed 
ClientError: InputConfiguration: Framework cannot load PyTorch model. No module named 'models'  For further troubleshooting common failures please visit: https://docs.aws.amazon.com/sagemaker/latest/dg/neo-troubleshooting-compilation.html

## Prepare model for deploying to Panorama device